In [1]:
import onnxruntime as ort
import numpy as np
import cv2

path="/media/paulo/b0f89c99-3dc8-428a-877d-75f05b66d7f8/home/paulo/fine-tune/out/"

In [24]:
!python -m samexporter.export_encoder --checkpoint ../../checkpoints/sam_vit_b_01ec64.pth \
    --output ../../checkpoints/sam_vit_b_preprocess.onnx \
    --model-type vit_b \
    --use-preprocess

/home/paulo/Desktop/ic/img2seq/.venv/lib/python3.10/site-packages/samexporter/mobile_encoder/tiny_vit_sam.py:795: UserWarning: Overwriting tiny_vit_5m_224 in registry with samexporter.mobile_encoder.tiny_vit_sam.tiny_vit_5m_224. This is because the name being registered conflicts with an existing name. Please check if this is not expected.
  return register_model(fn_wrapper)
/home/paulo/Desktop/ic/img2seq/.venv/lib/python3.10/site-packages/samexporter/mobile_encoder/tiny_vit_sam.py:795: UserWarning: Overwriting tiny_vit_11m_224 in registry with samexporter.mobile_encoder.tiny_vit_sam.tiny_vit_11m_224. This is because the name being registered conflicts with an existing name. Please check if this is not expected.
  return register_model(fn_wrapper)
/home/paulo/Desktop/ic/img2seq/.venv/lib/python3.10/site-packages/samexporter/mobile_encoder/tiny_vit_sam.py:795: UserWarning: Overwriting tiny_vit_21m_224 in registry with samexporter.mobile_encoder.tiny_vit_sam.tiny_vit_21m_224. This is bec

In [71]:
ort_sess = ort.InferenceSession(
    '../../checkpoints/sam_vit_b_preprocess.onnx',
)

In [74]:
from os import mkdir
from os.path import join, exists, basename
from glob import glob

dataset_path  = "/media/paulo/b0f89c99-3dc8-428a-877d-75f05b66d7f8/home/paulo/fine-tune/poly-bean/"
test_path = join(dataset_path, "test")
train_path = join(dataset_path, "train")

In [ ]:
for dir in [test_path, train_path]:
    images_path = glob(join(dir, "images", "*.jpg"))
    embeddings_path = join(dir, "embeddings")

    if not exists(embeddings_path):
        mkdir(embeddings_path)
    
    for image_path in images_path:
        image_name = basename(image_path)[:-4]
        image = cv2.imread(image_path)
        
        emb = ort_sess.run(None, {"input_image": image.astype(np.float32)})[0]
        out_path = join(embeddings_path, f"{image_name}.npy")
        with open(out_path, 'wb') as f:
            np.save(f, emb)


In [ ]:
# from os import listdir
# from os.path import join
# from glob import glob
# import matplotlib.pyplot as plt
# %matplotlib inline

# dirs = listdir(path)
# dirs.sort()

# for dir in dirs:
#     images_paths = glob(join(path, dir, "*.jpg"))
#     print(images_paths)
#     for image_path in images_paths:
#         image = cv2.imread(image_path)
#         image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

#         image = cv2.resize(image, (768, 1024), interpolation = cv2.INTER_AREA)
#         image = cv2.copyMakeBorder(image, 0, 0, 0, 256, cv2.BORDER_CONSTANT, None, (128, 128, 128))

#         plt.imshow(image)
#         break
#     break